# Gene set analysis using MAGMA and PoPS analysis

Magma scores where computed and processed at CCC, using bash. Starting point: NodoBIO/pryectos/genpsych/Resultados_Ines

### Workflow:
- MAGMA scores: Gene annotation of all SNPs present in BD GWAS summary statistics and gene-based analysis
- MAGMA analysis: Gene set analysis using GO, HPO and hallmark categories and gene property analysis using WGCNA modules
- PoPS: Polygenic Priority Score based on MAGMA gene based output and a feature matrix.

## Generate MAGMA scores
This step was run at CCC server, file paths not corresponding to repository.

In [ ]:
## Files preparation
#Filter of GWAS BD summary statistics to include only the official markers ('rs'), those with classic ACTG alleles and imputation info > 0.6
!awk '$5 ~ /^[ACGT]$/ && $6 ~ /^[ACGT]$/ && $3 ~ /^rs/ && $8 > 0.6 {print $0}' PGC/bd_main > PGC/bd_main_filtered
#Loc file of target SNPs (all present in GWAS BD)
!awk '{print $2 "\t" $1 "\t" $3}' PGC/bd_main_filtered > target_snps.loc
#Prepare annotation file with ENSID column, derived from gene annotation file of PoPS (Weeks et al. (2020))
#To add missing DGE and chr X genes, the genes to add were placed in a table with BioMart (Ensembl hg19) and pasted inside the gene_annot_jun10.txt file
!awk '{print $1 "\t" $3 "\t" $4 "\t" $5 }' gene_annot_jun10_added.txt > magma_ensid.txt
!sed '1d' magma_ensid.txt> magma_ensid.gene.loc #it was necessary to add genes on chrX

## Gene annotation
!module load plink/1.07
!module load magma/1.09

!magma --annotate window=5 --snp-loc pops-master/myfiles/target_snps.loc --gene-loc pops-master/myfiles/magma_ensid.gene.loc --out pops-master/myfiles/bd_genes

## Gene based analysis
!magma --bfile MAGMA/g1000_eur --gene-annot pops-master/myfiles/bd_genes.genes.annot --pval PGC/bd_main_filtered use='SNP','P' N=413466 --gene-model multi=snp-wise --out pops-master/myfiles/bd-scores


## MAGMA analysis
### Gene set analysis for enriched categories

The objective of this step is to validate the enriched categories that come from both GSEA results and ORA results (from WGCNA significant modules)

In [ ]:
!module load plink/1.07
!module load magma/1.09

!magma --gene-results pops-master/myfiles/bd-scores.genes.raw --set-annot MAGMA/validate_sets col=2,1 --out MAGMA/validate_sets_results

### Gene property analysis for WGCNA modules

The purpose of this step is to know if the gene coexpression modules are somehow related to the GWAS summary statistics, thus, if the significantly correlated modules have a significant p value among the genome wide associaiton signals.

In [ ]:
!magma --gene-results pops-master/myfiles/bd-scores.genes.raw --gene-covar pops-master/myfiles/wgcna_features/raw/MM.tsv --out MAGMA/validate_modules

## PoPS

We are going to delete the last two p value columns from the .genes.out file, as only the p value of the desired model has to be taken into account.

### Munge features

Processes run in local from RNASeq_BipolarDisorder repository. The pathway features and the WGCNA will be analysed together.

In [2]:
python ./results_MAGMA_PoPS/munge_feature_directory.py --gene_annot_path ./results_MAGMA_PoPS/magma/custom_annot_allhg38_forPoPS.txt --feature_dir Y:/featuresPoPS/raw/ --save_prefix Y:/featuresPoPS/munged/all_hg38 --max_cols 5000 --nan_policy "zero"

### Run PoPS

In [ ]:
!python ./results_MAGMA_PoPS/pops.py --gene_annot_path ./results_MAGMA_PoPS/magma/custom_annot_allhg38_forPoPS.txt --feature_mat_prefix D:/features/munged/all_features_added_genes --num_feature_chunks 12 --magma_prefix ./results_MAGMA_PoPS/magma/gene_based/scz_noTFM/scz_genebased --out_prefix ./results_MAGMA_PoPS/pops_output/scz-pops-complete_noTFM

### Process results

We want to check the role of the DEGs and genes on significant coexpression modules according to magma and PoPS. To do so, a common object is made from which the target genes will be filtered in.

In [4]:
import pandas as pd
import numpy as np

description_genes_file = './results_MAGMA_PoPS/magma/gene_annot_jun10_added.txt'
added_genes_list = './results_MAGMA_PoPS/magma/added_genes.txt'
results_gene_file = './results_MAGMA_PoPS/pops_output/bd-pops-complete.preds'
magma_p_value_file = './results_MAGMA_PoPS/magma/gene_based/bd-scores.genes.out'
save_gene_file = './results_MAGMA_PoPS/pops_output/description_bd-pops-complete.preds'

genes_desc = pd.read_csv(description_genes_file, sep = '\t').iloc[:,0:2]
f_pred = pd.read_csv(results_gene_file, sep='\t', header=0)
magma_p_values = pd.read_csv(magma_p_value_file, sep ='\s+', header = 0).iloc[:,[0,8]].rename(columns = {'GENE': 'ENSGID'}).rename(columns ={'P_MULTI': 'magma_p_value'})

genes_pred =pd.merge(f_pred, genes_desc, on = ['ENSGID'], how = 'left')
genes_pred = pd.merge(genes_pred, magma_p_values, on =['ENSGID'], how='left')

cols = ['ENSGID','NAME','magma_p_value','PoPS_Score','feature_selection_gene']
genes_pred = genes_pred[cols]

added_genes = list(pd.read_csv(added_genes_list, sep = '\t').iloc[:,0])
genes_pred.loc[genes_pred['ENSGID'].isin(added_genes), 'PoPS_Score'] = 'NaN'
genes_pred.loc[genes_pred['ENSGID'].isin(added_genes), 'feature_selection_gene'] = 'NaN'

#genes_pred.to_csv(save_gene_file, header = True, index = None, sep = '\t', mode = 'w')

In [5]:
DGE_file = './results_DGE/0.05_sig_padj.tsv'
DGE_table_scores_file = './results_MAGMA_PoPS/pops_output/DGE_scores_table.txt'
WGCNA_file = './results_WGCNA/genes_in_modules/genes_in_modules_ENSGID.txt'
WGCNA_table_scores_file = './results_MAGMA_PoPS/pops_output/WGCNA_scores_table.txt'

target_DGE = pd.read_csv(DGE_file, sep = '\t', header = 0).iloc[:,[0,1,8]]
target_DGE.columns = ['ENSGID', 'symbol','p_adj_DGE']

target_DGE = pd.merge(target_DGE, genes_pred, on = ['ENSGID'], how = 'left')
target_DGE.to_csv(DGE_table_scores_file, header = True, index = None, sep = '\t', mode = 'w')

target_WGCNA = pd.read_csv(WGCNA_file, sep = '\t', header = 0)
target_WGCNA = pd.merge(target_WGCNA, genes_pred, on = ['ENSGID'], how = 'left')
#target_WGCNA.to_csv(WGCNA_table_scores_file, header = True, index = None, sep = '\t', mode = 'w')

In [ ]:

#NOT USED for TFM: Feature description file derived from TODO: meter link del 2020.09.08.20190561-3.gz


#Load feature description file
import pandas as pd
import numpy as np
description_feature_file = './pops-input/feature_description.txt'
results_file = './pops-out/bd-pops.coefs'
save_file = './pops-out/bd-pops-desc.coefs'

f_description = pd.read_csv(description_feature_file, sep='\t', lineterminator='\n', header=0)
#print(f_description.iloc[0:9])
#Load coefficients file
f_coefs = pd.read_csv(results_file, sep='\t', header=0)
coefs= f_coefs.drop(index = [0,1,2])
coefs = coefs.rename(columns = {'parameter': 'featureID'})
#print(coefs.iloc[0:9])
#Merge and sort
coefs_desc = pd.merge(coefs, f_description, on = ['featureID'], how = 'left')
coefs_desc = coefs_desc.sort_values(by = ['beta'], ascending=False)
coefs_desc.to_csv(save_file, header=True, index=None, sep='\t', mode='w')
print(coefs_desc.iloc[0:9])